In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app

In [3]:
modeller = msm.convert('alanine_dipeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, constraints=app.HBonds, nonbondedMethod=app.NoCutoff)

In [10]:
explorer = oe.Explorer(topology, system, platform='CUDA')
explorer.set_coordinates(positions)

In [11]:
velocities = explorer.get_velocities()

In [12]:
(velocities**2).sum()

Quantity(value=0.0, unit=nanometer**2/(picosecond**2))

In [13]:
explorer.context.getState(getEnergy=True).getKineticEnergy()

Quantity(value=0.4280270110647903, unit=kilojoule/mole)

In [8]:
explorer.get_temperature()

Quantity(value=2.018811817084811, unit=kelvin)

In [ ]:
explorer.context.getState(getVelocities=True, getEnergy=True).getKineticEnergy()

In [ ]:
aa= explorer.get_velocities()

In [ ]:
for ii in range(22):
    print(aa[ii,:])

In [ ]:
explorer.n_dof/3

In [ ]:
explorer.n_atoms

In [ ]:
explorer.md.langevin(10000)

In [ ]:
temperature = 500.0*unit.kelvin
timestep = 2.0*unit.femtoseconds
collision_rate = 64.0/unit.picoseconds # 64 Openmm sugests 1 ps-1 but will it be the same for implicit solvent?
explorer.md.langevin.set_parameters(temperature=temperature, timestep=timestep,
                                    collision_rate=collision_rate)

threshold = 0.1*unit.angstroms

time_langevin_cicle = 1.0*unit.picoseconds
steps_langevin_cicle = int(time_langevin_cicle/timestep)
n_attempts_per_minimum = 250

minima_coordinates = []
minima_potential_energy = []
n_minima = 0

explorer.set_velocities_to_temperature(temperature)

coordinates = explorer.get_coordinates()
velocities = explorer.get_velocities()
explorer.quench.l_bfgs()

current_minimum_index = 0
current_minimum_coordinates = explorer.get_coordinates()
minima_coordinates.append(current_minimum_coordinates)
minima_potential_energy.append(explorer.get_potential_energy())
n_minima += 1

explorer.set_coordinates(coordinates)
explorer.set_velocities(velocities)

n_attempts = 0
n_cicles = 0

while n_attempts<n_attempts_per_minimum:
    
    pre_coordinates=coordinates
    explorer.md.langevin(steps_langevin_cicle)
    coordinates = explorer.get_coordinates()
    velocities = explorer.get_velocities()
    explorer.quench.l_bfgs()
    
    
    if explorer.distance.least_rmsd(current_minimum_coordinates) <= threshold:
        
        explorer.set_coordinates(coordinates)
        explorer.set_velocities(velocities)
        
    else:
        
        n_attempts+=1
        
        new_minimum=True
        for minimum_index in range(n_minima):
            if minimum_index!=current_minimum_index:
                if explorer.distance.least_rmsd(minima_coordinates[minimum_index]) <= threshold:
                    new_minimum = False
                    break
        if new_minimum:
            minima_coordinates.append(explorer.get_coordinates())
            minima_potential_energy.append(explorer.get_potential_energy())
            n_minima+=1
        
        coordinates=pre_coordinates
        explorer.set_coordinates(coordinates)
        explorer.set_velocities_to_temperature(temperature)

        print(n_attempts, n_cicles, n_minima)
        
    n_cicles+=1
        

In [ ]:
explorer.md.langevin.get_time()

In [ ]:
minima_potential_energy

Podemos calcular la distancia entre configuraciones para hacer un mapa por proximidad: mds o red con threshold.

In [ ]:
explorer.get_velocities()